**Using sqoop, import orders table into hdfs to folders /user/cloudera/problem1/orders. \
File should be loaded as Avro File and use snappy compression**

In [ ]:
# sqoop import \
# > -Dmapreduce.job.user.classpath.first=true
# > --connect jdbc:mysql://ms.itversity.com/retail_db \
# > --username retail_user \
# > --password itversity \
# > --table orders \
# > -m 2 \
# > --compression-codec 'snappy' \
# > --as-avrodatafile \
# > --target-dir /user/indirameduri/problem1/orders \
# > --delete-target-dir
# '''

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Arun3').master('yarn').getOrCreate()

In [ ]:
spark.sql('use indirameduri')

In [ ]:
order_items = spark.read.table('order_items')
order_items.show()

In [ ]:
spark.sql('select * from order_items').show()

In [ ]:

orders = spark.read.schema("order_id int ,order_date timestamp,order_cust_id int,order_status string").csv("/public/retail_db/orders")
#('/public/retail_db/orders/',header=True,inferSchema=True)

In [ ]:
orders.show()
orders.printSchema()

**Find total orders and total amount per status per day. The result should be sorted by order date in descending, order status in ascending and total amount in descending and total orders in ascending.**

In [ ]:
import pyspark.sql.functions
spark.sql('select distinct date_format(order_date,"EE") Day from orders').show()

In [ ]:
import pyspark.sql.functions
orders.createOrReplaceTempView('orders')
order_items.createOrReplaceTempView('orderitems')
data = spark.sql('''select  count(o.order_id) count,round(sum(oi.order_item_subtotal),2) total ,o.order_status,date_format(o.order_date,'yyyy-mm-dd') date from orders o join orderitems oi
                    on o.order_id = oi.order_item_order_id group by date,o.order_status order by date desc, o.order_status, total desc, count''')

In [ ]:
data.show(50,False)

In [ ]:
#parquet with gzip
data.coalesce(2).write.option('compression','gzip').mode('overwrite').parquet('/user/indirameduri/arun')


In [ ]:
#parquet with snappy
data.coalesce(2).write.option('compression','snappy').parquet('/user/indirameduri/arun/parquetsnappy')

In [ ]:
#csv with no compression
data.coalesce(3).write.option('compression','none').csv('/user/indirameduri/arun/csvnocompress')

In [ ]:
#write data to hive table

data.write.saveAsTable('arun')